In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModel
import torch
from pytorch_tabular.models.tab_transformer import TabTransformerModel
from tqdm import tqdm
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Step 1: Load Dataset
def load_data(file_path):
    """Loads the dataset and provides an initial overview."""
    data = pd.read_csv(file_path)
    print("Initial Dataset Info:\n", data.info())
    print("\nSample Data:\n", data.head())
    return data
data = load_data("C:\\Users\\sriha\\Music\\Case Comps\\NEST\\Data\\category_updated.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257577 entries, 0 to 257576
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  257577 non-null  int64  
 1   NCT Number                  257577 non-null  object 
 2   Study Title                 257577 non-null  object 
 3   Study Status                257577 non-null  object 
 4   Brief Summary               257577 non-null  object 
 5   Conditions                  257577 non-null  object 
 6   Primary Outcome Measures    247086 non-null  object 
 7   Secondary Outcome Measures  185779 non-null  object 
 8   Other Outcome Measures      18272 non-null   object 
 9   Sponsor                     257577 non-null  object 
 10  Collaborators               83679 non-null   object 
 11  Sex                         257317 non-null  object 
 12  Age                         257577 non-null  object 
 13  Phases        

In [3]:
data= data.head(5)
data.columns

Index(['Unnamed: 0', 'NCT Number', 'Study Title', 'Study Status',
       'Brief Summary', 'Conditions', 'Primary Outcome Measures',
       'Secondary Outcome Measures', 'Other Outcome Measures', 'Sponsor',
       'Collaborators', 'Sex', 'Age', 'Phases', 'Enrollment', 'Funder Type',
       'Study Type', 'Study Design', 'Start Month', 'Start Quarter',
       'Condition Category', 'Conditions_Category'],
      dtype='object')

In [3]:
"""Extracts embeddings for text data using the specified BERT model."""
biotokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
biomodel = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")

In [5]:
def bio_extract_text_embeddings(text_list, batch_size=512):
    """Extracts embeddings for text data using BioBERT with batch processing on GPU."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False)
    
    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        inputs = biotokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        
        with torch.no_grad():  # No gradient tracking for inference
            outputs = biomodel(**inputs)
        
        # Compute mean pooling of token embeddings (Sentence Representation)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(batch_embeddings)
    
    return np.vstack(embeddings)

In [4]:
# Combine all text attributes into a single column 'Unstructured'
data["Unstructured"] = data[
    ["Brief Summary", "Study Title", "Primary Outcome Measures", "Secondary Outcome Measures"]
].astype(str).agg(" [SEP] ".join, axis=1)

# Drop the original text columns
data = data.drop(columns=["Brief Summary", "Study Title", "Primary Outcome Measures", "Secondary Outcome Measures"])

# Display updated DataFrame
print(data)

        Unnamed: 0   NCT Number   Study Status  \
0                0  NCT00559130      Completed   
1                1  NCT00937664  Not_Completed   
2                2  NCT00441597      Completed   
3                3  NCT03296228      Completed   
4                4  NCT00421603      Completed   
...            ...          ...            ...   
257572      257572  NCT02360800      Completed   
257573      257573  NCT02352506      Completed   
257574      257574  NCT04996381      Completed   
257575      257575  NCT00380640      Completed   
257576      257576  NCT01844336      Completed   

                                               Conditions  \
0       Acute Respiratory Distress Syndrome|Acute Lung...   
1         Cancer|Solid Tumors|Advanced Solid Malignancies   
2       Ischemia Reperfusion Injury|Cardiovascular Dis...   
3                         Adolescent Idiopathic Scoliosis   
4                                      Cocaine Dependence   
...                              

In [17]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [14]:
data =data.head()

In [5]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader

# Load BioBERT model & tokenizer
biotokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
biomodel = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1").to("cuda")  # Move model to GPU

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to extract BioBERT embeddings in GPU-friendly batches
def bio_extract_text_embeddings(text_list, batch_size=512):
    """Extracts embeddings for text data using BioBERT with batch processing on GPU."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False)

    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        try:
            # Tokenization and moving inputs to GPU
            inputs = biotokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items()}  # Ensure all tensors are on GPU
            
            with torch.no_grad():  # No gradient tracking for inference
                outputs = biomodel(**inputs)

            # Compute mean pooling of token embeddings (Sentence Representation)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)  # Stays on GPU

            embeddings.append(batch_embeddings)

            # Free GPU memory after every batch
            del inputs, outputs
            torch.cuda.empty_cache()

        except torch.cuda.OutOfMemoryError:
            print("\n⚠️ CUDA OOM Error: Reducing batch size and retrying...\n")
            torch.cuda.empty_cache()
            return bio_extract_text_embeddings(text_list, batch_size=max(batch_size // 2, 1))  # Reduce batch size & retry

    return torch.cat(embeddings, dim=0).cpu().numpy()  # Move embeddings to CPU & convert to NumPy

# Split dataset into 100 chunks
num_chunks = 100
chunk_size = len(data) // num_chunks

for i in range(num_chunks):
    print(f"\n🚀 Processing chunk {i+1}/{num_chunks}...")

    # Select subset of data for this chunk
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(data)  # Last chunk gets remaining data
    chunk = data.iloc[start_idx:end_idx].copy()  # Use .copy() to avoid warnings

    # Convert text column to list
    bio_text_data = chunk["Unstructured"].astype(str).tolist()

    # Extract embeddings using batch processing on GPU
    embeddings_cpu = bio_extract_text_embeddings(bio_text_data, batch_size=512)  # Returns NumPy array

    # Create DataFrame with embedding columns
    bioembedding_df = pd.DataFrame(embeddings_cpu, index=chunk.index)
    bioembedding_df.columns = [f"Unstructured_embed_{j}" for j in range(embeddings_cpu.shape[1])]

    # Merge embeddings with original text in the chunk
    chunk = pd.concat([chunk, bioembedding_df], axis=1)

    # Save the chunk with both text and embeddings
    output_filename = f"NEST_chunk_{i+1}.csv"
    chunk.to_csv(output_filename, index=True)
    
    print(f"✅ Saved chunk {i+1} to {output_filename}")

    # Free GPU memory after each chunk
    del embeddings_cpu, bioembedding_df, chunk
    torch.cuda.empty_cache()

print("\n🎉 All chunks successfully saved as separate CSV files!")


Using device: cuda

🚀 Processing chunk 1/100...


Processing Batches: 100%|██████████| 6/6 [02:22<00:00, 23.70s/batch]


✅ Saved chunk 1 to NEST_chunk_1.csv

🚀 Processing chunk 2/100...


Processing Batches: 100%|██████████| 6/6 [02:19<00:00, 23.25s/batch]


✅ Saved chunk 2 to NEST_chunk_2.csv

🚀 Processing chunk 3/100...


...

✅ Saved chunk 98 to NEST_chunk_98.csv

🚀 Processing chunk 99/100...


Processing Batches: 100%|██████████| 6/6 [02:16<00:00, 22.76s/batch]


✅ Saved chunk 99 to NEST_chunk_99.csv

🚀 Processing chunk 100/100...


Processing Batches: 100%|██████████| 6/6 [02:17<00:00, 22.84s/batch]


✅ Saved chunk 100 to NEST_chunk_100.csv

🎉 All chunks successfully saved as separate CSV files!


In [8]:
bioembedding_df_1 = pd.read_csv('NEST_embed_chunk_1.csv',)
bioembedding_df_1

,Unnamed: 0,Unstructured_embed_0,Unstructured_embed_1,Unstructured_embed_2,Unstructured_embed_3,Unstructured_embed_4,Unstructured_embed_5,Unstructured_embed_6,Unstructured_embed_7,Unstructured_embed_8,...,Unstructured_embed_758,Unstructured_embed_759,Unstructured_embed_760,Unstructured_embed_761,Unstructured_embed_762,Unstructured_embed_763,Unstructured_embed_764,Unstructured_embed_765,Unstructured_embed_766,Unstructured_embed_767
0,0,-0.173068,-0.077015,-0.525726,-0.037300,-0.203847,0.030538,-0.040831,0.19147,0.245301,...,-0.253266,0.127309,-0.11036,0.348753,-0.166995,0.212118,0.242659,-0.229306,-0.052761,-0.388725
1,1,-0.199231,-0.065837,-0.115439,-0.000676,-0.122404,0.003565,-0.092257,-0.12714,0.089963,...,-0.047347,0.167718,0.06174,0.211448,0.024834,0.416763,0.272169,-0.318060,0.294743,-0.060007
2,2,-0.173068,-0.077015,-0.525726,-0.037300,-0.203847,0.030538,-0.040831,0.19147,0.245301,...,-0.253266,0.127309,-0.11036,0.348753,-0.166995,0.212118,0.242659,-0.229306,-0.052761,-0.388725
3,3,-0.199231,-0.065837,-0.115439,-0.000676,-0.122404,0.003565,-0.092257,-0.12714,0.089963,...,-0.047347,0.167718,0.06174,0.211448,0.024834,0.416763,0.272169,-0.318060,0.294743,-0.060007
4,4,-0.173068,-0.077015,-0.525726,-0.037300,-0.203847,0.030538,-0.040831,0.19147,0.245301,...,-0.253266,0.127309,-0.11036,0.348753,-0.166995,0.212118,0.242659,-0.229306,-0.052761,-0.388725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1360,1360,-0.173068,-0.077015,-0.525726,-0.037300,-0.203847,0.030538,-0.040831,0.19147,0.245301,...,-0.253266,0.127309,-0.11036,0.348753,-0.166995,0.212118,0.242659,-0.229306,-0.052761,-0.388725
1361,1361,-0.199231,-0.065837,-0.115439,-0.000676,-0.122404,0.003565,-0.092257,-0.12714,0.089963,...,-0.047347,0.167718,0.06174,0.211448,0.024834,0.416763,0.272169,-0.318060,0.294743,-0.060007
1362,1362,-0.173068,-0.077015,-0.525726,-0.037300,-0.203847,0.030538,-0.040831,0.19147,0.245301,...,-0.253266,0.127309,-0.11036,0.348753,-0.166995,0.212118,0.242659,-0.229306,-0.052761,-0.388725
1363,1363,-0.199231,-0.065837,-0.115439,-0.000676,-0.122404,0.003565,-0.092257,-0.12714,0.089963,...,-0.047347,0.167718,0.06174,0.211448,0.024834,0.416763,0.272169,-0.318060,0.294743,-0.060007


In [19]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader

# Load BioBERT model & tokenizer
biotokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
biomodel = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1").to("cuda")  # Move model to GPU

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to extract BioBERT embeddings in GPU-friendly batches with memory control
def bio_extract_text_embeddings(text_list, batch_size=256):
    """Extracts embeddings for text data using BioBERT with batch processing on GPU while handling OOM errors."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False)

    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        try:
            # Tokenization and moving inputs to GPU
            inputs = biotokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
            inputs = {key: val.to(device) for key, val in inputs.items()}  # Move all tensors to GPU

            with torch.no_grad():  # No gradient tracking for inference
                outputs = biomodel(**inputs)

            # Compute mean pooling of token embeddings (Sentence Representation)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1)  # Keep on GPU
            
            embeddings.append(batch_embeddings)

            # Free GPU memory after each batch
            del inputs, outputs
            torch.cuda.empty_cache()

        except torch.cuda.OutOfMemoryError:
            print("\n⚠️ CUDA OOM Error: Reducing batch size to avoid crash.\n")
            torch.cuda.empty_cache()
            return bio_extract_text_embeddings(text_list, batch_size=max(batch_size // 2, 1))  # Reduce batch size & retry

    return torch.cat(embeddings, dim=0).cpu().numpy()  # Ensure output is on CPU

# Split dataset into 15 chunks
num_chunks = 15
chunk_size = len(data) // num_chunks

for i in range(num_chunks):
    print(f"\n🚀 Processing chunk {i+1}/{num_chunks}...")

    # Select subset of data for this chunk
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i != num_chunks - 1 else len(data)  # Last chunk gets remaining data
    chunk = data.iloc[start_idx:end_idx]

    # Convert text column to list
    bio_text_data = chunk["Unstructured"].astype(str).tolist()

    # Extract embeddings using batch processing on GPU
    embeddings_cpu = bio_extract_text_embeddings(bio_text_data, batch_size=256)  # Dynamically adjusts batch size

    # Create DataFrame with embedding columns
    bioembedding_df = pd.DataFrame(embeddings_cpu, index=chunk.index)
    bioembedding_df.columns = [f"Unstructured_embed_{i}" for i in range(embeddings_cpu.shape[1])]

    # Save each chunk separately to avoid memory overload
    output_filename = f"NEST_embed_chunk_{i+1}.csv"
    bioembedding_df.to_csv(output_filename, index=True)
    
    print(f"✅ Saved chunk {i+1} to {output_filename}")

    # Free GPU memory after each chunk
    del embeddings_cpu, bioembedding_df
    torch.cuda.empty_cache()

print("\n🎉 All embeddings successfully saved in separate CSV files!")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [15]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader

# Load BioBERT model & tokenizer
biotokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
biomodel = AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1").to("cuda")  # Move model to GPU

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Function to extract BioBERT embeddings in GPU-friendly batches
def bio_extract_text_embeddings(text_list, batch_size=512):
    """Extracts embeddings for text data using BioBERT with batch processing on GPU."""
    embeddings = []
    
    dataloader = DataLoader(text_list, batch_size=batch_size, shuffle=False)

    for batch_texts in tqdm(dataloader, desc="Processing Batches", unit="batch"):
        # Tokenization and moving inputs to GPU
        inputs = biotokenizer(batch_texts, return_tensors="pt", truncation=True, padding=True, max_length=512)
        inputs = {key: val.to(device) for key, val in inputs.items()}  # Ensure all tensors are on GPU
        
        with torch.no_grad():  # No gradient tracking for inference
            outputs = biomodel(**inputs)
        
        # Compute mean pooling of token embeddings (Sentence Representation)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)  # Stays on GPU
        
        embeddings.append(batch_embeddings)

    return torch.cat(embeddings, dim=0)  # Returns a single tensor on GPU

# Process 'Unstructured' text attribute in 512-size batches
attribute = "Unstructured"
print(f"Processing attribute: {attribute}")

bio_text_data = data[attribute].astype(str).tolist()

# Extract embeddings using batch processing on GPU
embeddings_tensor = bio_extract_text_embeddings(bio_text_data, batch_size=512)  # Returns tensor on GPU

# Move embeddings back to CPU for DataFrame creation
embeddings_cpu = embeddings_tensor.cpu().numpy()

# Create DataFrame with embedding columns
bioembedding_df = pd.DataFrame(embeddings_cpu, index=data.index)
bioembedding_df.columns = [f"{attribute}_embed_{i}" for i in range(embeddings_cpu.shape[1])]

# Drop the original attribute column and merge embeddings
data = data.drop(columns=[attribute])
data = pd.concat([data, bioembedding_df], axis=1)

print("Embeddings successfully replaced the original attributes.")
data.to_csv('NEST_embed.csv')

Using device: cuda
Processing attribute: Unstructured


Processing Batches:   0%|          | 0/49 [00:01<?, ?batch/s]


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [75]:
data.to_csv('NEST_embed.csv')

,Unnamed: 0,NCT Number,Study Status,Conditions,Other Outcome Measures,Sponsor,Collaborators,Sex,Age,Phases,...,Unstructured_embed_758,Unstructured_embed_759,Unstructured_embed_760,Unstructured_embed_761,Unstructured_embed_762,Unstructured_embed_763,Unstructured_embed_764,Unstructured_embed_765,Unstructured_embed_766,Unstructured_embed_767
0,0,NCT00559130,Completed,Acute Respiratory Distress Syndrome|Acute Lung...,NaN,"MedaSorb Technologies, Inc",NaN,ALL,"ADULT, OLDER_ADULT",NaN,...,-0.098257,0.107153,-0.193596,0.135309,0.092266,-0.020007,-0.063864,0.123716,-0.106573,0.031290
1,1,NCT00937664,Not_Completed,Cancer|Solid Tumors|Advanced Solid Malignancies,NaN,AstraZeneca,NaN,ALL,"ADULT, OLDER_ADULT",PHASE1,...,0.018887,0.115186,-0.080183,0.219760,0.035297,0.034310,0.078921,-0.039460,-0.072698,-0.054465
2,2,NCT00441597,Completed,Ischemia Reperfusion Injury|Cardiovascular Dis...,NaN,Radboud University Medical Center,Pfizer,MALE,ADULT,PHASE4,...,-0.112456,0.116638,-0.116839,0.092405,-0.204348,0.031370,-0.016791,0.158138,-0.017895,0.021559
3,3,NCT03296228,Completed,Adolescent Idiopathic Scoliosis,NaN,The University of Hong Kong,"AO Foundation, AO Spine",ALL,"CHILD, ADULT",NaN,...,-0.013137,-0.063034,0.019267,-0.061944,-0.091516,-0.104973,0.211876,0.175831,-0.115984,-0.035802
4,4,NCT00421603,Completed,Cocaine Dependence,NaN,New York State Psychiatric Institute,National Institute on Drug Abuse (NIDA),ALL,ADULT,PHASE2,...,-0.024513,0.121777,0.032171,0.037329,-0.005663,-0.278086,0.064402,0.270871,-0.001445,0.048874


In [26]:
clinicaltokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
clinicalmodel = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

c:\Users\sriha\anaconda3\envs\nest\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sriha\.cache\huggingface\hub\models--emilyalsentzer--Bio_ClinicalBERT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [27]:
def clinical_extract_text_embeddings(text_data):

    embeddings = []
    for text in text_data:
        inputs = clinicaltokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        outputs = clinicalmodel(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())

    return np.vstack(embeddings)


In [32]:
data = data.head(2)
data

,Unnamed: 0,NCT Number,Study Title,Study Status,Brief Summary,Conditions,Primary Outcome Measures,Secondary Outcome Measures,Other Outcome Measures,Sponsor,...,Age,Phases,Enrollment,Funder Type,Study Type,Study Design,Start Month,Start Quarter,Condition Category,Conditions_Category
0,0,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,Completed,The hypothesis of this study is use of CytoSor...,Acute Respiratory Distress Syndrome|Acute Lung...,Relative IL-6 levels as a percent (%) of basel...,"Ventilator Free Days, Reduction cytokines TNF-...",NaN,"MedaSorb Technologies, Inc",...,"ADULT, OLDER_ADULT",NaN,100.0,INDUSTRY,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,11,4,Other Rare or Unclassified,Other Rare or Unclassified
1,1,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,Not_Completed,The primary purpose of this study is to find o...,Cancer|Solid Tumors|Advanced Solid Malignancies,Assessment of adverse events (based on CTCAE v...,Pharmacokinetic effect of AZD7762 when adminis...,NaN,AstraZeneca,...,"ADULT, OLDER_ADULT",PHASE1,24.0,INDUSTRY,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,7,3,Oncology,Oncology


In [33]:
for attribute in tqdm(['Primary Outcome Measures', 'Secondary Outcome Measures'], desc="Processing Attributes"):
    print(f"Processing attribute: {attribute}")
    
    # Convert text column to string
    clinical_text_data = data[attribute].astype(str).tolist()

    # Extract embeddings and ensure proper NumPy array structure
    embeddings = clinical_extract_text_embeddings(bio_text_data)
    
    # Convert to torch tensor and move to device (ensure dtype compatibility)
    clinical_text_data_tensor = torch.tensor(embeddings, dtype=torch.float32).to(device)

    # Move embeddings back to CPU for DataFrame creation
    clinicalembeddings_cpu = clinical_text_data_tensor.cpu().numpy()

    # Create DataFrame with embedding columns
    clinicalembedding_df = pd.DataFrame(clinicalembeddings_cpu, index=data.index)
    clinicalembedding_df.columns = [f"{attribute}_embed_{i}" for i in range(embeddings_cpu.shape[1])]

    # Drop the original attribute column and merge embeddings
    data = data.drop(columns=[attribute])
    data = pd.concat([data, clinicalembedding_df], axis=1)

print("Embeddings successfully replaced the original attributes.")

Processing Attributes:   0%|          | 0/2 [00:00<?, ?it/s]

Processing attribute: Primary Outcome Measures


Processing Attributes:   0%|          | 0/2 [05:39<?, ?it/s]


KeyboardInterrupt: 

In [51]:
# Step 4: Combine Features and Embeddings
# Combine structured features with embeddings
X_combined = np.hstack((numerical_scaled, categorical_encoded, bio_embeddings, clinical_embeddings))
y = data['Study Status']  # Target column

# Train-Test Split for Combined Data
X_train_combined, X_test_combined, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)



NameError: name 'clinical_embeddings' is not defined

In [ ]:
# Step 5: TabTransformer for Combined Data
def train_tab_transformer(X_train, y_train):
    """Trains a TabTransformer model on the combined structured and text data."""
    model = TabTransformerModel(
        task="classification",
        input_dim=X_train.shape[1],
        n_classes=len(np.unique(y_train)),
        cat_idxs=[],
        cat_dims=[],
        n_dnn_layers=2,
        n_dnn_units=64,
        learning_rate=1e-3,
        batch_size=32,
        epochs=10
    )
    model.fit(X_train, y_train)
    return model

# Train TabTransformer
tab_transformer_model = train_tab_transformer(X_train_combined, y_train)

In [ ]:
# Step 6: Evaluate TabTransformer
predictions = tab_transformer_model.predict(X_test_combined)
print("TabTransformer Classification Report:\n", classification_report(y_test, predictions))